# Data API

## Imports

In [2]:
import urllib.request, json
import pandas as pd
import math
import pickle
import collections
from tqdm import tqdm
import geopandas
import datetime
import os
import numpy as np

## Global Variables

In [3]:
START_DATE = '01062019'
END_DATE = '15062022'
MAX_LAT = 52.6
MAX_LONG = -0.32
MIN_LAT = 52.5
MIN_LONG = -0.33
QUALITY_THRESHOLD = 90
NORMALISE_DF = True

## Initial Site Retrieval

In [4]:
# Download information about all UK sites
url_text = "https://webtris.highwaysengland.co.uk/api/v1/sites"
with urllib.request.urlopen(url_text) as url:
    data = json.loads(url.read().decode())
sites = data['sites']
sites

[{'Id': '1',
  'Name': 'MIDAS site at M4/2295A2 priority 1 on link 105009001; GPS Ref: 502816;178156; Westbound',
  'Description': 'M4/2295A2',
  'Longitude': -0.520379557723297,
  'Latitude': 51.4930115367112,
  'Status': 'Inactive'},
 {'Id': '2',
  'Name': 'MIDAS site at A1M/2259B priority 1 on link 126046101; GPS Ref: 514029;294356; Southbound',
  'Description': 'A1M/2259B',
  'Longitude': -0.320275451712423,
  'Latitude': 52.5351577963853,
  'Status': 'Active'},
 {'Id': '3',
  'Name': 'MIDAS site at M5/7482B priority 1 on link 109001601; GPS Ref: 388120;253057; Northbound',
  'Description': 'M5/7482B',
  'Longitude': -2.17513825916014,
  'Latitude': 52.1756517888158,
  'Status': 'Active'},
 {'Id': '4',
  'Name': 'MIDAS site at M3/2173A priority 1 on link 103009101; GPS Ref: 442770;118058; Westbound',
  'Description': 'M3/2173A',
  'Longitude': -1.39237366736779,
  'Latitude': 50.9603586282494,
  'Status': 'Inactive'},
 {'Id': '5',
  'Name': 'MIDAS site at M25/5764B priority 1 on li

## Helper Functions

In [5]:
def get_quality_area(sites,
                     max_lat=math.inf,
                     max_long=math.inf,
                     min_lat=-math.inf,
                     min_long=-math.inf,
                     start_date='01062021',
                     end_date = '15062022',
                     quality_threshold = 90):
    '''
    Returns a dataframe of Traffic Count sites in the specified area and time with sufficent reporting quality
            Parameters:
                    max_lat, max_long, min_lat, min_long (int): Coordinates defining the rectangular area of interest. Default is entire globe.
                    start_date, end_date (str): Strings of the start and end dates of our search ddmmyy
                    quality_threshold (int): Only indclude sites that have at least quality_threshold % of times reporting data
            Returns:
                    quality_area_sites_df (dataframe): Report high quality sites, cols are: Id, Name, Description, Longitude, Latitude, Status
    '''
    # Convert sites query into df and filter onto our area
    sites_df = pd.DataFrame(data = sites)
    area_sites_df = sites_df.loc[(min_long < sites_df.Longitude) & (sites_df.Longitude < max_long)
                                & (min_lat < sites_df.Latitude) & (sites_df.Latitude < max_lat)]
    area_sites_df = area_sites_df.reset_index(drop=True)
    area_ids = list(area_sites_df.Id)
    
    # Next filter onto sites with good quality data:
    quality_responces = []
    for site_id in tqdm(area_ids):
        url_text = f"https://webtris.highwaysengland.co.uk/api/v1/quality/overall?sites={site_id}&start_date={start_date}&end_date={end_date}"
        with urllib.request.urlopen(url_text) as url:
            responce = json.loads(url.read().decode())
        quality_responces.append(responce)
        
    # We only want sites with quality greater than threshold
    good_quality_ids = []
    for responce in quality_responces:
        if responce['data_quality'] >= quality_threshold:
            good_quality_ids.append(responce['sites'])

    quality_area_sites_df = area_sites_df.loc[area_sites_df.Id.isin(good_quality_ids)]
    quality_area_sites_df = quality_area_sites_df.reset_index(drop=True)
    
    print('quality_area_sites_df: {}'.format(quality_area_sites_df))
    return quality_area_sites_df

In [6]:
def daily_report_query_url(site_id, page_num, start_date = '15062021', end_date = '15062022'):
    '''Generates the query url for page page_num of traffic reporting of site site_id'''
    query_url = f"https://webtris.highwaysengland.co.uk/api/v1/reports/Daily?sites={site_id}&start_date={start_date}&end_date={end_date}&page={page_num}&page_size=10000"
    return query_url

In [7]:
def get_site_report(site_id, start_date='15062021', end_date='15062022'):
    '''
    Returns a dataframe of traffic counts on a specified site and date range.
            Parameters:
                    site_id (str): Site's unique id
                    start_date, end_date (str): Strings of the start and end dates of our search ddmmyy
            Returns:
                    report_df (dataframe): Report of traffic counts for that site
                    header (dict): Columns of the dataframe
    '''
    # Download page 1
    report_url = daily_report_query_url(site_id, 1, start_date, end_date)
    with urllib.request.urlopen(report_url) as url:
        report_page = json.loads(url.read().decode())
        
    # Work out how many pages are required    
    header = report_page['Header']
    rows = report_page['Rows']
    row_count = header['row_count']
    total_pages = math.ceil(row_count / 10000)
    # Make a dataframe of the rows so dar
    report_df = pd.DataFrame(data = rows)
    
    for i in range(2, total_pages+1):
        # Get page i of the report
        report_url = daily_report_query_url(site_id, i, start_date, end_date)
        with urllib.request.urlopen(report_url) as url:
            report_page = json.loads(url.read().decode())
        
        rows = report_page['Rows']
        current_page_df = pd.DataFrame(data = rows)
        report_df = pd.concat([report_df, current_page_df], ignore_index=True)

    return report_df, header

In [8]:
def get_reports_from_sites_df(sites_df, start_date, end_date):
    '''
    Returns a dataframe of traffic counts for an entire set of sites
            Parameters:
                    sites_df (dataframe): The sites we want to query, has the same columns as get_quality_area function's output
                    start_date, end_date (str): Strings of the start and end dates of our search ddmmyy
            Returns:
                    report_df (dataframe): Report of traffic counts for the sites
    '''
    # Get the reports on the site
    train_reports =  collections.defaultdict(str)
    # Go through all the site ids and get reports
    print("sites_df: {}".format(sites_df))
    for site_id in tqdm(sites_df.Id):
        report, header = get_site_report(site_id, start_date, end_date)
        report['site_id'] = site_id
        train_reports[site_id] = report
        
    # Combine reports into one df
    report_df = pd.concat(list(train_reports.values()), ignore_index=True)
    return report_df

In [9]:
def clean_report(report_df):
    '''
    Cleans the traffic count report with a few key steps:
    1. Format the column names and remove redundant columns
    2. Converts the count columns into intergers
    3. Remove rows with blank data
    4. Remove rows that only report one value (zero)
    5. Add a timestamp column to the report
    
            Parameters:
                    report_df (dataframe): The report of traffic count data outputted by get_reports_from_sites_df
            
            Returns:
                    clean_report_df (datafrane): The cleaned report
    '''
    # Step 1.
    clean_col_names = [
        'site_name',
        'report_date',
        'time_period_ending',
        'time_interval',
        '0-520cm',
        '521-660cm',
        '661-1160cm',
        '1160+cm',
        '0-10mph',
        '11-15mph',
        '16-20mph',
        '21-25mph',
        '26-30mph',
        '31-35mph',
        '36-40mph',
        '41-45mph',
        '46-50mph',
        '51-55mph',
        '56-60mph',
        '61-70mph',
        '71-80mph',
        '80+mph',
        'avg_mph',
        'total_volume',
        'site_id']
    report_df.columns = clean_col_names
    clean_cols = [
         'site_name',
         'site_id',
         'report_date',
         'time_period_ending',
         'time_interval',
         '0-520cm',
         '521-660cm',
         '661-1160cm',
         '1160+cm',
         'avg_mph',
         'total_volume']
    clean_report_df = report_df[clean_cols]
    
    # Steps 2., 3., 4.
    interger_cols = [
         '0-520cm',
         '521-660cm',
         '661-1160cm',
         '1160+cm',
         'total_volume']
    def remove_rows(df):
        df = df.loc[df['total_volume'] != '']  # Remove empty rows
        x = df.groupby('site_id')['total_volume'].nunique()
        zero_sites = list(x[x==1].index)  # Remove sites where the volume is always zero
        df = df.loc[~df.site_id.isin(zero_sites)]
        df[interger_cols] = df[interger_cols].astype('int32')
        return df
    clean_report_df = remove_rows(clean_report_df)
    
    # Step 5.
    def get_timestamp(row):
        year, month,day = row['report_date'].split('T')[0].split('-')
        hour, minute, second = row['time_period_ending'].split(':')
        return datetime.datetime(int(year),int(month),int(day), int(hour), int(minute))
    
    clean_report_df['timestamp'] = clean_report_df.apply(get_timestamp,axis=1)
    return clean_report_df

In [10]:
# Function used to normalsise the count data
def normalise(clean_report):
    interger_cols = ['0-520cm', '521-660cm', '661-1160cm', '1160+cm', 'total_volume']
    for name in interger_cols:
        new_name = f"{name}_normalised"
        # for ever row in the report present the row's site id's mean volume
        mean = clean_report.groupby('site_id')[name].transform("mean")
        # normalise
        clean_report.loc[:, new_name] = clean_report[name] / mean
        # filter so we don't have rows with a small mean which causes a pole
    return clean_report[mean>1]

In [11]:
# A pipeline of stages for downloading and normalising reporting.
def download_clean_pipeline(start_date, end_date, max_lat, max_long, min_lat, min_long, quality_threshold = 90, normalise_df=True):
    print('Producing clean report df using following parameters:')
    print('------------------------------------------')
    print('start_date: {}'.format(start_date))
    print('end_date: {}'.format(end_date))
    print('max_lat: {}'.format(max_lat))
    print('max_long: {}'.format(max_long))
    print('min_lat: {}'.format(min_lat))
    print('min_long: {}'.format(min_long))
    print('quality_threshold: {}'.format(quality_threshold))
    print('normalise: {}'.format(normalise_df))
    print('------------------------------------------')

    # Get the quality data
    sites_df = get_quality_area(sites, max_lat, max_long, min_lat, min_long, start_date, end_date, quality_threshold)
    # Download the report
    report_df = get_reports_from_sites_df(sites_df, start_date, end_date)
    # Clean the report
    clean_report_df = clean_report(report_df)
    # Normalsie the report
    if normalise:
        clean_report_df_norm = normalise(clean_report_df)
        return clean_report_df_norm
    else:
        return clean_report_df

## Ungrouped Data

In [12]:
clean_report_df = download_clean_pipeline(START_DATE, END_DATE, MAX_LAT, MAX_LONG, MIN_LAT, MIN_LONG,  QUALITY_THRESHOLD, NORMALISE_DF)
clean_report_df.head()

Producing clean report df using following parameters:
------------------------------------------
start_date: 01062019
end_date: 15062022
max_lat: 52.6
max_long: -0.32
min_lat: 52.5
min_long: -0.33
quality_threshold: 90
normalise: True
------------------------------------------


100%|██████████| 3/3 [00:00<00:00,  5.73it/s]


quality_area_sites_df:      Id                                               Name Description  \
0     2  MIDAS site at A1M/2259B priority 1 on link 126...   A1M/2259B   
1  4466  MIDAS site at A1M/2259A priority 1 on link 199...   A1M/2259A   
2  7810  TMU Site 6783/1 on link A1 southbound exit for...      6783/1   

   Longitude   Latitude  Status  
0  -0.320275  52.535158  Active  
1  -0.320531  52.535018  Active  
2  -0.320749  52.535875  Active  
sites_df:      Id                                               Name Description  \
0     2  MIDAS site at A1M/2259B priority 1 on link 126...   A1M/2259B   
1  4466  MIDAS site at A1M/2259A priority 1 on link 199...   A1M/2259A   
2  7810  TMU Site 6783/1 on link A1 southbound exit for...      6783/1   

   Longitude   Latitude  Status  
0  -0.320275  52.535158  Active  
1  -0.320531  52.535018  Active  
2  -0.320749  52.535875  Active  


100%|██████████| 3/3 [01:42<00:00, 34.11s/it]


,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
0,A1M/2259B,2,2019-06-01T00:00:00,00:14:00,0,24,6,6,16,62,52,2019-06-01 00:14:00,0.259554,0.234373,0.445363,0.564074,0.325197
1,A1M/2259B,2,2019-06-01T00:00:00,00:29:00,1,17,3,7,20,60,47,2019-06-01 00:29:00,0.183851,0.117186,0.519590,0.705092,0.293928
2,A1M/2259B,2,2019-06-01T00:00:00,00:44:00,2,23,3,3,17,61,46,2019-06-01 00:44:00,0.248740,0.117186,0.222681,0.599329,0.287674
3,A1M/2259B,2,2019-06-01T00:00:00,00:59:00,3,12,2,0,19,59,33,2019-06-01 00:59:00,0.129777,0.078124,0.000000,0.669838,0.206375
4,A1M/2259B,2,2019-06-01T00:00:00,01:14:00,4,14,5,3,16,61,38,2019-06-01 01:14:00,0.151407,0.195311,0.222681,0.564074,0.237644


In [13]:
clean_report_df.describe()

,0-520cm,521-660cm,661-1160cm,1160+cm,total_volume,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
count,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000,316194.000000
mean,77.271586,18.194061,9.375542,20.397825,125.238676,1.000000,1.000000,1.000000,1.000000,1.000000
std,75.031979,21.118919,10.680899,19.356200,113.564047,0.867800,0.944294,1.015015,0.755598,0.774218
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,3.000000,2.000000,6.000000,31.000000,0.205481,0.205593,0.222681,0.381704,0.265487
50%,55.000000,8.000000,5.000000,13.000000,87.000000,0.812390,0.703118,0.556801,0.763408,0.872315
75%,120.000000,31.000000,14.000000,32.000000,207.000000,1.600887,1.602277,1.645197,1.586458,1.619731
max,580.000000,158.000000,81.000000,142.000000,716.000000,6.546909,7.782487,8.908821,6.790712,5.040551


## Preprocessing and saving data

### Save full clean report

In [14]:
print("Counting na values: {}".format(clean_report_df.isna().sum()))
os.makedirs('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data', exist_ok=True)  
clean_report_df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/clean_report.csv')  

Counting na values: site_name                  0
site_id                    0
report_date                0
time_period_ending         0
time_interval              0
0-520cm                    0
521-660cm                  0
661-1160cm                 0
1160+cm                    0
avg_mph                    0
total_volume               0
timestamp                  0
0-520cm_normalised         0
521-660cm_normalised       0
661-1160cm_normalised      0
1160+cm_normalised         0
total_volume_normalised    0
dtype: int64


### Save site_id, timestamp

In [30]:
def dt64_to_float(dt64):
    """
    year = dt64.astype('M8[Y]')
    print("year: {}".format(year))
    days = (dt64 - year).astype('timedelta64[D]')
    print("days: {}".format(days))
    year_next = year + np.timedelta64(1, 'Y')
    days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
    print("days_of_year: {}".format(days_of_year))
    dt_float = 1970 + year.astype(float) + days / (days_of_year)
    print("dt_float: {}".format(dt_float))
    """
    dt_float = dt64.values.astype('float64')
    dt_float = ((dt_float)-dt_float.min())/(dt_float.max()-dt_float.min())
    print(dt_float)
    return dt_float

In [16]:
os.makedirs('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data', exist_ok=True)  
timestamp_df = pd.DataFrame(clean_report_df[['site_id', 'timestamp']], index=clean_report_df.index)

timestamp_df['timestamp'] = dt64_to_float(timestamp_df['timestamp'])
timestamp_df['timestamp'] = ((timestamp_df['timestamp'])-timestamp_df['timestamp'].min())/(timestamp_df['timestamp'].max()-timestamp_df['timestamp'].min())

timestamp_df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/timestamp.csv')  

[1.55934804e+18 1.55934894e+18 1.55934984e+18 ... 1.65533574e+18
 1.65533664e+18 1.65533754e+18]


### Save site_id, avg_mph

In [17]:
os.makedirs('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data', exist_ok=True)  
avg_mph_df = pd.DataFrame(clean_report_df[['site_id', 'avg_mph']], index=clean_report_df.index)
avg_mph_df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv')  

### Save site_id, total_volume

In [18]:
os.makedirs('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data', exist_ok=True)  
total_volume_df = pd.DataFrame(clean_report_df[['site_id', 'total_volume']], index=clean_report_df.index)
total_volume_df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/total_volume.csv')  

### Save site_id, total_volume_normalised

In [19]:
os.makedirs('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data', exist_ok=True)  
total_volume_normalised_df = pd.DataFrame(clean_report_df[['site_id', 'total_volume_normalised']], index=clean_report_df.index)
total_volume_normalised_df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/total_volume_normalised.csv')  

## Grouped Data

In [42]:
# Groupby site_id and report_date
dfs = [site_day for _, site_day in clean_report_df.groupby(['site_id', 'report_date'])]

print(len(dfs))
"""
dfs_site_daily_count = []
for i, df in enumerate(dfs):
    print(i, df['site_id'])
    if i > 5:
        break
    daily_count = df['total_volume'].sum()
    df['daily_count'] = daily_count
    #df.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/{}_{}.csv'.format(df['site_id'], df['report_date']))  
"""

dfs[0].head()
df_site_day = dfs[0]
df_site_day['daily_count'] = df_site_day['total_volume'].sum()

df_site_day['report_date'] = pd.to_datetime(df_site_day['report_date'], infer_datetime_format=True)
df_site_day['month'] = df_site_day['report_date'].dt.month
df_site_day['day'] = df_site_day['report_date'].dt.day
df_site_day['hour'] = df_site_day['timestamp'].dt.hour

# timestamp preprocessing
df_site_day['timestamp_min_max'] = dt64_to_float(df_site_day['timestamp'])
df_site_day.to_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/report_daily_count.csv')  

df_site_day.head()

3310
[0.         0.01052632 0.02105263 0.03157895 0.04210526 0.05263158
 0.06315789 0.07368421 0.08421053 0.09473684 0.10526316 0.11578947
 0.12631579 0.13684211 0.14736842 0.15789474 0.16842105 0.17894737
 0.18947368 0.2        0.21052632 0.22105263 0.23157895 0.24210526
 0.25263158 0.26315789 0.27368421 0.28421053 0.29473684 0.30526316
 0.31578947 0.32631579 0.33684211 0.34736842 0.35789474 0.36842105
 0.37894737 0.38947368 0.4        0.41052632 0.42105263 0.43157895
 0.44210526 0.45263158 0.46315789 0.47368421 0.48421053 0.49473684
 0.50526316 0.51578947 0.52631579 0.53684211 0.54736842 0.55789474
 0.56842105 0.57894737 0.58947368 0.6        0.61052632 0.62105263
 0.63157895 0.64210526 0.65263158 0.66315789 0.67368421 0.68421053
 0.69473684 0.70526316 0.71578947 0.72631579 0.73684211 0.74736842
 0.75789474 0.76842105 0.77894737 0.78947368 0.8        0.81052632
 0.82105263 0.83157895 0.84210526 0.85263158 0.86315789 0.87368421
 0.88421053 0.89473684 0.90526316 0.91578947 0.92631579 0

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,...,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised,daily_count,month,day,hour,timestamp_min_max
0,A1M/2259B,2,2019-06-01,00:14:00,0,24,6,6,16,62,...,0.259554,0.234373,0.445363,0.564074,0.325197,16392,6,1,0,0.000000
1,A1M/2259B,2,2019-06-01,00:29:00,1,17,3,7,20,60,...,0.183851,0.117186,0.519590,0.705092,0.293928,16392,6,1,0,0.010526
2,A1M/2259B,2,2019-06-01,00:44:00,2,23,3,3,17,61,...,0.248740,0.117186,0.222681,0.599329,0.287674,16392,6,1,0,0.021053
3,A1M/2259B,2,2019-06-01,00:59:00,3,12,2,0,19,59,...,0.129777,0.078124,0.000000,0.669838,0.206375,16392,6,1,0,0.031579
4,A1M/2259B,2,2019-06-01,01:14:00,4,14,5,3,16,61,...,0.151407,0.195311,0.222681,0.564074,0.237644,16392,6,1,1,0.042105
